### Этап 1. Получение ссылок на книги автора

Для поиска всех книг автора мы будем пользоваться поисковой строкой сайта. Обратите внимание на формат URL-ов запросов:
```
https://www.chitai-gorod.ru/search/result.php?q={author}&type=author
```
где вместо `{author}` подставляется имя автора.

Рекомендуемые варианты значения для `{author}`: `Фрай М.`, `Хантер Э.`, `Емец Д.`.

**Замечание:** URL [не позволяет](https://ru.wikipedia.org/wiki/URL#%D0%9A%D0%BE%D0%B4%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_URL) использовать кириллические символы, поэтому для кодирования части URL-а, содержащей кириллицу, стоит воспользоваться функцией `urllib.parse.quote` с использованием, в данном случае, кодировки `windows-1251`.

Попробуйте пролистать [страничку](https://www.chitai-gorod.ru/search/result.php?q=%D4%F0%E0%E9%20%CC.&type=author) вниз, и вы увидите, что новые книги подгружаются динамически. В некоторый момент появляется кнопка "Показать ещё", нажав на которую можно дополнительно подгрузить выдачу.

Чтобы получить все ссылки на карточки с книгами, нужно подгрузить полную выдачу по запросу.

Существует несколько способов получить ссылку на карточку:
1. получить атрибут `data-product` одного из тегов с классом `product-card` и использовать знание о формате URL-ов страничек с книгами;
2. непосредственно найти ссылку.

In [371]:
import re
import requests
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
import urllib
import pandas as pd

## Don't work

In [372]:
%%time
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}

names_authors = ["Фрай М.", "Хантер Э.", "Емец Д."]
names_url = ['https://www.chitai-gorod.ru/search/result.php?q={}&type=author'.format(urllib.parse.quote(name, encoding="CP1251")) for name in names_authors]
t = ThreadPool(16)
r = t.map(lambda name_url: requests.get(name_url, headers=headers), names_url)
t.close()
t.join()

CPU times: user 81.6 ms, sys: 3.34 ms, total: 84.9 ms
Wall time: 314 ms


In [373]:
one = r[0].content
soup = BeautifulSoup(one, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" ng-app="search">
 <head>
  <link href="https://img-gorod.ru" rel="dns-prefetch"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="65c799ed1259b388" name="yandex-verification"/>
  <meta content="05725c0ecd32a193" name="yandex-verification"/>
  <meta content="9733050c10fc15ab" name="yandex-verification"/>
  <meta content="hA8vdpybpjnGvyu1SpXRYZo9iv068BCHmT7Y6QVfcSc" name="google-site-verification"/>
  <meta content="noyaca" name="robots"/>
  <meta content=" " name="keywords"/>
  <title>
   Результаты поиска
  </title>
  <link href="/bitrix/components/kav/subscribe/templates/.default/style.css?1497429415104" rel="stylesheet" type="text/css"/>
  <link href="/bitrix/components/kav/chitai.socialauth/templates/.default/style.css?14974294151301" rel="stylesheet" type="text/css"/>
  <!-- BEGIN JIVOSITE CODE -->
  <script type="text/javascript">
   (function(){ var wid

## Selenium
http://selenium-python.readthedocs.io/

https://kreisfahrer.gitbooks.io/selenium-webdriver/

In [374]:
from selenium import webdriver
import html
from time import sleep

In [375]:
with webdriver.Firefox() as driver:
    driver.get("https://mail.ru/")
    sleep(3)

In [376]:
url_current = names_url[0]
url_current

'https://www.chitai-gorod.ru/search/result.php?q=%D4%F0%E0%E9%20%CC.&type=author'

In [381]:
import random

def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)
    
def hasclass(driver, path):
    try:
        driver.find_element_by_class_name(path)
        return True
    except:
        return False
    
def get_books_info(driver, urls, t_sleep=1):
    result = []
    for url in urls:
        driver.get(url)
        random_sleep()
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        SCROLL_PAUSE_TIME = 2

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # [ Show more ]
                try: 
                    more = driver.find_element_by_class_name('js__show-more-cards')
                    print("Press 'Show_more' = ", haspath(driver,'js__show-more-cards'))
                    more.click()
                except:
                    break
                random_sleep(1, 1)
            last_height = new_height

        items = driver.find_elements_by_class_name("product-card")
        for item in items:
            book_url_info = {
                'id': item.get_attribute("data-product")}
        
            if book_url_info['id'] is None:
                continue
            book_url_info['id'] = html.unescape(book_url_info['id'])
            result.append(book_url_info)
        
    return result


with webdriver.Firefox() as driver:
    result = get_books_info(driver, names_url)

Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True


In [382]:
result = pd.DataFrame(result)
result.head()

,id
0,1085970
1,1068930
2,1059170
3,1051063
4,1021715


### Этап 2. Получение информации о книгах

Рассмотрим в качестве примера [карточку](https://www.chitai-gorod.ru/catalog/book/1059170/) книги Макса Фрая "Мертвый ноль".
<img width = '1080px' src="images/1059170.png">


На скриншоте выделены 4 области, из каждой области требуется извлечь следующие элементы:
1. поле "ID карточки";
2. поля "Название", "Автор", "Рейтинг", "Голоса";
3. поле "Цена";
4. вся таблица.

Дополнительно требуется создать поле "Обложка", которое будет хранить ссылку на обложку книги (расположена слева на скриншоте).

Таким образом карточка представима в виде следующего словаря:
```json
{
    "ID карточки": 1059170,
    "Название": "Мертвый ноль",
    "Автор": "Фрай М.",
    "Рейтинг": 4.0,
    "Голоса": 16,
    "Цена": 375,
    "Серия": "Сновидения Ехо",
    "Издательство": "АСТ",
    "Год издания": 2018,
    "Кол-во страниц": 448,
    "ISBN": "9785171089733",
    "Тираж": 30000,
    "Формат": "20.5 x 13 x 2.5",
    "Тип обложки": "Твердая бумажная",
    "Возраст": "16+",
    "ID товара": 2659485,
    "Обложка": "https://img-gorod.ru/upload/iblock/aec/aec2cfaece8a6190f319f1853cad7cf5.jpg"
}
```

Предположим, что у нас есть функция `extract_book_info(card_id)`, которая для карточки с номером `card_id` возвращает описанный выше словарь. Тогда требуемую таблицу можно получить следующим образом:
```python
import pandas as pd

result = list(map(extract_book_info, card_ids))

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

with open('data/hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)
```

Пример результата работы программы можно найти [здесь](hw_4_sample.csv).

## Beautiful Soup
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

<img width = '330px' src="images/bsoup.jpg">

In [383]:
from bs4 import BeautifulSoup

In [384]:
list_of_url = ["https://www.chitai-gorod.ru/catalog/book/{}/?watch_fromlist=search_result".
               format(id) for id in list(result.id.values)]

In [385]:
%%time
url = "https://www.chitai-gorod.ru/catalog/book/{}/?watch_fromlist=search_result".format(result.id[0])
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}

t = ThreadPool(16)
r = requests.get(url, headers=headers)
t.close()
t.join()

CPU times: user 32.2 ms, sys: 13.6 ms, total: 45.8 ms
Wall time: 614 ms


In [386]:
book_html = r.text
soup = BeautifulSoup(book_html, 'html.parser')

In [387]:
def re_sub(string):
    return re.sub("\s+", ' ', string.strip())
def re_sub_one(string):
    return re.sub("\s", ' ', string.strip()).split()[0]
def re_sub_two(string):
    string = re.sub("\s", ' ', string.strip()).split()[1]
    return re.sub("\D", '', string.strip())
def re_sub_n(string):
     return re.sub("\D", '', string.strip())


In [388]:
film_info = {
    "ID карточки": '',
    "Название": '',
    "Автор": '',
    "Рейтинг": '',
    "Голоса": '',
    "Цена": '',
    "Серия": '',
    "Издательство": '',
    "Год издания": '',
    "Кол-во страниц": '',
    "ISBN": '',
    "Тираж": '',
    "Формат": '',
    "Тип обложки": '',
    "Возраст": '',
    "ID товара": '',
    "Обложка": ''
}

In [389]:
film_info['ID карточки'] = result.id[0]
film_info['Название'] = re_sub(soup.find('h1', class_="product__title js-analytic-product-title").text)
film_info['Автор'] = re_sub(soup.find('a', class_= "link product__author").text)
film_info['Рейтинг'] = re_sub_one(soup.find('div',class_="rating").text)
film_info['Голоса'] = re_sub_two(soup.find('div',class_="rating").text)
film_info['Цена'] = re_sub_n(soup.find('div', class_='price').text)

film_info

{'ID карточки': '1085970',
 'Название': 'Тяжелый свет Куртейна. Синий',
 'Автор': 'Фрай М.',
 'Рейтинг': '5',
 'Голоса': '1',
 'Цена': '374',
 'Серия': '',
 'Издательство': '',
 'Год издания': '',
 'Кол-во страниц': '',
 'ISBN': '',
 'Тираж': '',
 'Формат': '',
 'Тип обложки': '',
 'Возраст': '',
 'ID товара': '',
 'Обложка': ''}

In [390]:
table = soup.find('div', class_='product__props')
rows = table.find_all('div', class_="product-prop")

In [391]:
data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('div'))) for row in rows]
print(data)
for property_book in data:
    if property_book[0] == "ID товара:":
        film_info["ID товара"] = property_book[1]
    else:
        film_info[property_book[0]] = property_book[1]
film_info

[['Серия', 'Другая сторона'], ['Издательство', 'АСТ'], ['Год издания', '2018'], ['Кол-во страниц', '352'], ['ISBN', '9785171121945'], ['ID товара:', '2689500']]


{'ID карточки': '1085970',
 'Название': 'Тяжелый свет Куртейна. Синий',
 'Автор': 'Фрай М.',
 'Рейтинг': '5',
 'Голоса': '1',
 'Цена': '374',
 'Серия': 'Другая сторона',
 'Издательство': 'АСТ',
 'Год издания': '2018',
 'Кол-во страниц': '352',
 'ISBN': '9785171121945',
 'Тираж': '',
 'Формат': '',
 'Тип обложки': '',
 'Возраст': '',
 'ID товара': '2689500',
 'Обложка': ''}

In [414]:
len(list(result.id.values))

440

In [418]:
%%time
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}
t = ThreadPool(16)
r = t.map(lambda id: (id, requests.get("https://www.chitai-gorod.ru/catalog/book/{}/?watch_fromlist=search_result".format(id), headers=headers)), list(result.id.values))
t.close()
t.join()

CPU times: user 13.8 s, sys: 790 ms, total: 14.6 s
Wall time: 24.4 s


In [421]:
# create def extract_book_info
def extract_book_info(book):
    book_html = book[1].text
    soup = BeautifulSoup(book_html, 'html.parser')
    film_info = {
    "ID карточки": '',
    "Название": '',
    "Автор": '',
    "Рейтинг": '',
    "Голоса": '',
    "Цена": '',
    "Серия": '',
    "Издательство": '',
    "Год издания": '',
    "Кол-во страниц": '',
    "ISBN": '',
    "Тираж": '',
    "Формат": '',
    "Тип обложки": '',
    "Возраст": '',
    "ID товара": '',
    "Обложка": '',
    "Переводчик": '',
    "Редактор": '',
    "Художник": ''
    }
    film_info['ID карточки'] = book[0]
    film_info['Название'] = re_sub(soup.find('h1', class_="product__title js-analytic-product-title").text)
    film_info['Автор'] = re_sub(soup.find('a', class_= "link product__author").text)
    film_info['Рейтинг'] = re_sub_one(soup.find('div',class_="rating").text)
    film_info['Голоса'] = re_sub_two(soup.find('div',class_="rating").text)
    try:
        film_info['Цена'] = re_sub_n(soup.find('div', class_='price').text)
    except:
        film_info['Цена'] = ''
    table = soup.find('div', class_='product__props')
    rows = table.find_all('div', class_="product-prop")
    data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('div'))) for row in rows]
    for property_book in data:
        if property_book[0] == "ID товара:":
            film_info["ID товара"] = property_book[1]
        else:
            film_info[property_book[0]] = property_book[1]
    return film_info

In [423]:
import pandas as pd
card = r
result = list(map(extract_book_info, card))

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

In [424]:
with open('hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [425]:
df.head()

,ID карточки,ID товара,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Обложка,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена
258,1002042,2607833,9785699973705,Емец Д.,12+,2017,0,Эксмо,416,Лед и пламя Тартара,,,Суворова Т.,0,Мефодий Буслаев. Легендарное детское фэнтези,Твердая бумажная,4000,21.8 x 14.5 x 2.5,,342
10,1004167,2608544,9785171053802,Фрай М.,18+,2017,4,АСТ,288,Новая чайная книга,,,,5,Миры Макса Фрая,Твердая бумажная,7000,20.6 x 13.2 x 2.3,,332
172,1004240,2608971,9785001111900,Хантер Э.,6+,2017,1,Абрис,350,Расколотый прайд,,Голосовская А.Ю.,,5,Земля Отважных,Твердая бумажная,7000,20.6 x 13 x 1.7,Насыров Л.Х.,285
257,1006687,2611099,9785699973712,Емец Д.,12+,2017,0,Эксмо,416,Первый эйдос,,,,0,Мефодий Буслаев. Легендарное детское фэнтези,Твердая бумажная,4000,21.2 x 13.5 x 2.3,,342
9,1007486,2611890,9785179827924,Фрай М.,16+,2017,14,АСТ,416,Отдай мое сердце,,,Кравченко Е.,4,Сновидения Ехо,Твердая бумажная,27000,20.7 x 13.3 x 2,,423
